Input (300, 1)
  ↓
Conv1D (64 filters, kernel=5, relu) → 특징 추출
  ↓
MaxPooling1D → 차원 축소
  ↓
Dropout(0.3) → 과적합 방지
  ↓
LSTM(64) → 시간 순서 정보 학습
  ↓
Dense(32, relu) → 고수준 특징 처리
  ↓
Dropout(0.2) → 과적합 방지
  ↓
Dense(1, sigmoid) → 부정맥 여부 출력

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ppg_train.npy to ppg_train.npy


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 1. 데이터 불러오기
X = np.load('ppg_train.npy')
y = np.array([0]*500 + [1]*500)  # 0=정상, 1=부정맥 (라벨 수에 따라 수정)
# 정상 500개, 부정맥 500개

In [ ]:
# 2. 정규화 스케일링 (0~1 범위로)
scaler = MinMaxScaler()
X_scaled = np.array([scaler.fit_transform(x.reshape(-1, 1)).flatten() for x in X])

In [ ]:
# 3. 차원 reshape (Conv1D와 LSTM 입력 차원: (samples, timesteps, features))
# (샘플 수, 시간, 채널) 구조
X_scaled = X_scaled.reshape(-1, 300, 1)  # 예: 각 샘플 300길이 신호

In [ ]:
# 4. 데이터 분할: 학습용 vs 검증용
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# 5. 모델 설계
model = Sequential([
    # CNN: 주파수 패턴
    Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(300, 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    # LSTM: 리듬의 순서 정보
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # 이진 분류  # 0 or 1 출력
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# 6. 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.4859 - loss: 0.6922 - val_accuracy: 0.5400 - val_loss: 0.6861
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.6034 - loss: 0.6592 - val_accuracy: 0.8350 - val_loss: 0.5389
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 91ms/step - accuracy: 0.7765 - loss: 0.5338 - val_accuracy: 0.8150 - val_loss: 0.4632
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.7808 - loss: 0.5075 - val_accuracy: 0.6300 - val_loss: 0.6092
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.7374 - loss: 0.5398 - val_accuracy: 0.8200 - val_loss: 0.4288
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - accuracy: 0.8043 - loss: 0.4960 - val_accuracy: 0.8350 - val_loss: 0.4051
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.8280 - loss: 0.4071 - val_accuracy: 0.8200 - val_loss: 0.4810
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.7705 - loss: 0.6669 - val_accuracy: 0.4900 

In [ ]:
# 7. 성능 평가
loss, acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {acc:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8079 - loss: 0.4877
Validation Accuracy: 0.8200
